In [2]:
# As usual, a bit of setup
import time, os, re, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from layers import *
from solver import Solver
from rnn import RNN
from data import *
from configure import *
from utils import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [19]:
%%time
splited = split_data()

CPU times: user 1min 48s, sys: 9.33 s, total: 1min 58s
Wall time: 2min 14s


In [16]:
N, T, D = 10, 20, 12
x = np.random.rand(N, T, D)
w = np.random.randn(D)
b = 1
out = x.reshape(N * T, D).dot(w).reshape(N, T) + b
#np.array([[1,2,3],[1,2,3]]).dot(np.array([1,2,3]))
dout = np.random.randn(*out.shape)
dx = dout.reshape(-1, 1).dot(w.reshape(1, -1)).reshape(x.shape)

In [98]:
out, cache = temporal_dot_forward(x, w, 1)
dx, dw, db = temporal_dot_backward(out, cache)

In [22]:
len(1)

TypeError: object of type 'int' has no len()

In [8]:
np.random.randn(6).dot(np.random.randn(6))

4.846899046117918

In [57]:
print(*[COLUMNS[i] for i in [SS_CP,PITH_CP,ROLL_CP,SS_FO,PITH_FO,ROLL_FO]], sep='\n')

['_SSTICK_CAPT', '_SSTICK_CAPT-1', '_SSTICK_CAPT-2', '_SSTICK_CAPT-3']
['_PITCH_CAPT_SSTICK', '_PITCH_CAPT_SSTICK-1', '_PITCH_CAPT_SSTICK-2', '_PITCH_CAPT_SSTICK-3']
['_ROLL_CAPT_SSTICK', '_ROLL_CAPT_SSTICK-1', '_ROLL_CAPT_SSTICK-2', '_ROLL_CAPT_SSTICK-3']
['_SSTICK_FO', '_SSTICK_FO-1', '_SSTICK_FO-2', '_SSTICK_FO-3']
['_PITCH_FO_SSTICK', '_PITCH_FO_SSTICK-1', '_PITCH_FO_SSTICK-2', '_PITCH_FO_SSTICK-3']
['_ROLL_FO_SSTICK', '_ROLL_FO_SSTICK-1', '_ROLL_FO_SSTICK-2', '_ROLL_FO_SSTICK-3']


In [45]:
sscp = flight_df[COLUMNS[SS_CP]].values
ssfo = flight_df[COLUMNS[SS_FO]].values
p_cp = flight_df[COLUMNS[PITH_CP]].values
p_fo = flight_df[COLUMNS[PITH_FO]].values
r_cp = flight_df[COLUMNS[ROLL_CP]].values
r_fo = flight_df[COLUMNS[ROLL_FO]].values

a=sscp * p_cp + ssfo * p_fo
b=p_cp+p_fo

In [44]:
a[np.arange(40), abs(a).argmax(axis=1)]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.35156   ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.01073503,  0.92284501,  0.        , -1.31834996,  0.        ,
        0.        ,  0.        ,  0.92284501,  0.043945  ,  0.        ,
        1.09862494,  0.        ,  0.        ,  2.85642505,  0.        ,
       -2.94431496, -4.04294014, -4.04294014, -7.82220984, -9.00872517])

In [46]:
b[np.arange(40), abs(b).argmax(axis=1)]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.70311999,
       -0.35156   ,  0.        , -1.274405  , -0.57128501, -1.23046005,
       -1.09862494,  0.        ,  0.        ,  1.18651497,  1.40623999,
        1.01073503,  1.31834996, -1.36229503, -1.31834996, -0.43945   ,
        0.        ,  0.74706501,  0.92284501,  1.14257002,  1.09862494,
        1.31834996,  0.        ,  0.        ,  2.85642505, -0.043945  ,
       -2.94431496, -4.04294014, -4.04294014, -7.82220984, -9.00872517])

In [29]:
b

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.01073503,  1.31834996,  0.        , -1.23046005,  0.        ,
        0.        ,  0.        ,  0.92284501,  1.14257002,  0.        ,
        1.31834996,  0.        ,  0.        ,  2.85642505,  0.        ,
        0.        , -1.45018494,  0.52733999, -1.49413002, -0.26367   ])

In [21]:
cor_vrtg = pd.read_csv(PATH+'../VrtgCorrectors.csv')
files_set = set(f.split('_')[-1] for f in os.listdir(PATH))
cor_set = set(cor_vrtg['FILENAME'])

In [23]:
files_set - cor_set

set()

{'161107003552653.csv',
 '160618155148602.csv',
 '160409171442817.csv',
 '160511051823827.csv',
 '160902082202652.csv',
 '161029043118601.csv',
 '161102112337650.csv',
 '160611055244817.csv',
 '160914044005653.csv',
 '160721061144626.csv',
 '161111033055653.csv',
 '161106085338626.csv',
 '160807091007690.csv',
 '161030013231817.csv',
 '161007111139650.csv',
 '160818035629653.csv',
 '160903021246651.csv',
 '160601022316817.csv',
 '160909034922626.csv',
 '160511060210690.csv',
 '160826033139827.csv',
 '160906034408626.csv',
 '160913080551826.csv',
 '161105041521650.csv',
 '160917005225626.csv',
 '160802082929817.csv',
 '161113101914651.csv',
 '161103120810601.csv',
 '160930041240826.csv',
 '161017054840653.csv',
 '160903043019653.csv',
 '160924051329826.csv',
 '161104041300651.csv',
 '160808130523652.csv',
 '160706130252601.csv',
 '161114145841650.csv',
 '160612151801601.csv',
 '160815052756690.csv',
 '160810021004817.csv',
 '161028093813653.csv',
 '161018162724826.csv',
 '16111613362169